In [1]:
import numpy as np
import pandas as pd
import geopandas
import random
import matplotlib.pyplot as plt
import math
import seaborn as sns

from mip import *
import timeit

from stable_baselines3 import DQN

In [2]:
# Dataset and models
filename_base = 'dataset/dual_s_base.csv' #filename of data with only 1 user
# model_lin = DQN.load("edge_agent_30_lin")
# model_exp = DQN.load("edge_agent_30_exp")
model_act = DQN.load("edge_agent_action")
model_und = DQN.load("edge_agent_under_train")
model_prop = DQN.load("edge_agent_proper_train")

model_thres10 = DQN.load("edge_agent_thres10")

users_high = 19
users_low = 1 #to calculate the users numbers from action
users_res = 2

### Utility Function to Load EUA Data

In [3]:
#Add data to users
def add_data(series): #not used now
    random_speed = random.uniform(0,1)
    if random_speed <= 0.3:
        series['Speed']= 0
    if random_speed > 0.3 and random_speed <= 0.6:   
        series['Speed']= random.uniform(1, 2)
    if random_speed > 0.6:
        series['Speed'] = random.uniform(10, 20)
    series['Direction']= random.randrange(0, 360, 10)
    qospref = [ random.randint(1,3), random.randint(1,3) ]
    qospref.sort()
    series['LowQoS'] = qospref[0]
    series['HighQoS'] = qospref[1]
    return series

#================Load Planet Lab data
def load_planetlab():
    #Convert to triangle
    ldata = np.loadtxt('PlanetLabData_1')[np.tril_indices(490)]
    ldata = ldata[ ldata != 0]
    ldata =np.unique(ldata)
    np.set_printoptions(suppress=True)
    length = ldata.shape[0]
    latency_row = 150
    latency_col = (length//latency_row) #Global Data used
    ldata = np.resize(ldata, latency_col*latency_row)
    latency = ldata.reshape(latency_row,-1)
    return latency

#=================Fetch Network latency
def fetch_network_lat(distance, latency_data):
    rep_lat = np.random.choice(latency_data[distance], size=1, replace=True)
    return rep_lat/1000 #convert latency to seconds
    

#===============User Data
def load_users(num_of_users):
    user_raw = pd.read_csv("eua/users.csv")
    user_raw = user_raw.rename_axis("UID")
    df = user_raw.sample(num_of_users)
#     user_raw = user_raw.reset_index()
    gdf = geopandas.GeoDataFrame(df, geometry = geopandas.points_from_xy(df.Longitude, df.Latitude), crs = {'init': 'epsg:4326'})
    user = gdf [['geometry']] #Keep Geometry column
    user = user.to_crs(epsg=28355) #Covert the format of data
    #Insert additional data to dataframe
    #user = user.apply(add_data, axis=1)
    return user

#================Server Data
def load_servers(num_of_servers):
    server_raw = pd.read_csv("eua/servers.csv")
    server_raw = server_raw.rename_axis("SID")
    df = server_raw.sample(num_of_servers) #Sample Servers Randomly
    gdf = geopandas.GeoDataFrame(df, geometry = geopandas.points_from_xy(df.LONGITUDE, df.LATITUDE), crs = {'init': 'epsg:4326'})
    server = gdf [['geometry']] #Keep Geometry column
    server = server.to_crs(epsg=28355) #Cover to crs in Australian EPSG
    def add_radius(series):
#         radius = random.randrange(150, 250, 10)
        radius = 150 #radus fixed to 150
        series.geometry = series.geometry.buffer(radius)
        series['radius'] = radius
#         series['resource'] = tcomp
        return series
    server = server.apply(add_radius, axis = 1)
    return server

#================neighbourhood Computing
def ngb_matrix(U, N, S): 
    #U: number of users
    #N: number of servers
    #S: number of services
    # U X N matrix
    user = load_users(U)
    server = load_servers(N)
    neighbourhood = np.zeros([U, N]) #matrix user(row) vs server(col)
    network_latency = np.zeros(N) #network latency for each server
    latency_data = load_planetlab() #load planetlad data, return matrix of bin size 150
    
    for u in range(0, U):
        for n in range(0, N):
            if server.iloc[n].geometry.contains(user.iloc[u].geometry):
                neighbourhood[u,n]=1
                #compute distance and assign latency
                distance = server.iloc[n].geometry.centroid.distance(user.iloc[u].geometry)
                rep_lat = fetch_network_lat(int(distance), latency_data) #fetch latency according to distance
                if network_latency[n] < rep_lat:
                    network_latency[n] = rep_lat
                
            else:
                neighbourhood[u,n]=0
    
    service = np.zeros(U)
    
    #assign service to the users
    for u in range(0, U):
        service[u] = random.randrange(0, S, 1) #generate a random service request to the user
        
    #find service request in each server
    server_service = np.zeros((N, S))
    
    #find the number of service request in each server
    for n in range(0, N):
        for u in range(0, U):
            if neighbourhood[u][n] == 1:
                server_service[n][int(service[u])] += 1
                
                
    
    return neighbourhood, user, server, service, server_service, network_latency

#Plot the user and server dataset on the map
def plot_data(user, server):
    %config InlineBackend.figure_format='retina'
    %matplotlib inline

    cbd = geopandas.read_file('eua/maps', crs = {'init': 'epsg=28355'} ) #read cbd-australia location data

    fig, ax = plt.subplots(1, 1, figsize=(15,10))
    ax.set_aspect('equal')

    ax.set_xlim(319400, 322100)
    ax.set_ylim(5811900, 5813700)

    user.plot(ax=ax, marker='o', color='red', markersize=20, zorder=3, label="users")
    server.plot(ax =ax, linestyle='dashed', edgecolor='green', linewidth=1, facecolor="none", zorder=1)
    server.centroid.plot(ax=ax, marker='s', color='blue', markersize=50, zorder=2, label="server")
    cbd.plot(ax=ax, color='grey', zorder=0, alpha = 0.3);

    ax.set_title("MEC Environment(EUA): CBD Melbourne(Australia)")
    ax.legend(bbox_to_anchor=(1, 0), loc='lower left')
    plt.show()

## Load data for 1 user

In [4]:
def generate_server_state(num_server):
    df = pd.read_csv(filename_base)
    
#     df['ram'] = df['ram'].div(1000).round(0).astype(int)
#     df['workload_cpu'] = df['workload_cpu'].div(10).round(0).astype(int)
    df['workload_gpu'] = df['workload_gpu'].multiply(1/80).round(0).astype(int) #round gpu workload
#     df['users_yolo'] = df['users_yolo'].div(100).round(0).astype(int)
#     df['users_mnet'] = df['users_mnet'].div(100).round(0).astype(int)
    
    
    #get unique data in set
    ram = df.ram.unique()
    cores = df.cores.unique()
    workload_cpu = df.workload_cpu.unique()

    
    server_state = []
    gamma = []
    
    for s_id in range(num_server):
        #generate unique state
        gram = np.random.choice(ram, 1)[0]
        gcores = np.random.choice(cores, 1)[0]
        gwl_c = np.random.choice(workload_cpu, 1)[0]
    
        
       
        #fetch gamma for the state
        fetch_state = df.loc[ (df['ram'] == gram) & (df['cores']== gcores) & (df['workload_cpu']==gwl_c) ]
       
        gwl_g = fetch_state.sample().iloc[0]['workload_gpu'] #fetch workload randmoly
        fetch_time = fetch_state.loc[ (df['workload_gpu'] == gwl_g) ]
        
        time_yolo = fetch_time['time_yolo'].mean() #average of time for particular state
        time_mnet = fetch_time['time_mnet'].mean()
        
        gs1 = server_service[s_id][0] #assignt the state according to service request to each server
        gs2 = server_service[s_id][1] 
        server_state.append( [gram, gcores, gwl_c, gwl_g, gs1, gs2] )
        
        gamma.append((time_yolo, time_mnet)) #append the gamma value of each server
    
    return server_state, gamma

# Algorithms

### ILP Algorithm

In [5]:
def ilp_algo():
    ## ===================================ILP with python mip
    # >> solver_name=GRB
    # >> Currently using CBC
    I = range(U) #user
    J = range(N) #server

    alloc = Model(sense=MAXIMIZE, name="alloc")

    def coverage(user_ix, server_ix):
        if ngb[user_ix][server_ix]==1:
            return 1
        else:
            return 0

    #U: num of users, N: num of servers
    x = [[ alloc.add_var(f"x{i}{j}", var_type=BINARY) for j in J] for i in I]

    #Objective Equation
    alloc.objective = xsum( x[i][j]  for i in I for j in J )

    #1 .Coverage Constraint
    for i in I:
        for j in J:        
                if not coverage(i,j):
                    alloc += x[i][j] == 0

    #2. User allocation to single server constrain
    for i in I:
        alloc += xsum( x[i][j] for j in J ) <=1

    #3. Latency Constraint
    for j in J:
        alloc += xsum( gamma[j][int(service[i])]*x[i][j] for i in I ) <=latency_threshold-network_latency[j] 

    #alloc.write("test-model.lp")

    #===========Start Optimization=========
    alloc.optimize(max_seconds=25)

    #==========ILP Ends here
    #print(f"Number of Solutions:{qoe.num_solutions}")
    ilp_allocation = [ (i,j) for i in I for j in J if x[i][j].x >= 0.99]

    #print(f"Number of Solutions:{qoe.num_solutions}")
    #print(f"Objective Value:{qoe.objective_value}")
    allocated_num_users = len(ilp_allocation)
    print("ILP Allocated Num of Users: {}".format(allocated_num_users))
    # selected.sort()
    return ilp_allocation

### RL Allocation Algorithm

In [6]:
#Load model
def rl_algo_prop():
    #for each server find the number of possible users using model
    server_capacity = np.zeros((N, S))
    for server_id in range(N):
        state = server_state[server_id]
        
        action = model_prop.predict(np.array(state), deterministic=True)
        
        u1 = (action[0]//10)*2 + 1
        u2 = (action[0]%10)*2 + 1
        server_capacity[server_id][0] = u1 #model output
        server_capacity[server_id][1] = u2 #model output
    
    #sort with number of servers per user
    col1 = np.array([np.sum(ngb,axis=1)])
    col2 = np.array([np.arange(U)])
    sorted_ngb = np.concatenate((ngb, col1.T, col2.T), axis=1) #add rowsum and index column
    sorted_ngb = sorted_ngb[np.argsort(sorted_ngb[:, N])] #sort the rows based on rowsum column

    #allocation heuristic
    #run allocation algorithm
    rl_allocation = []

    for i in range(U):
        server_list = np.where(sorted_ngb[i, :N] == 1)[0] #get the list of server to which user is connected

        if len(server_list) == 0: #skip users with no server
            continue
        
        ser = int(service[i]) #which service user is requesting
        choosen_server = server_list[np.argmax(server_capacity[server_list, ser])] #find the id of choosen server
        
        
        if  server_capacity[choosen_server][ser] > 0: #assigning user to choosen_server
            server_capacity[choosen_server][ser] -= 1 #decrement the server capacity

            rl_allocation.append( (int(sorted_ngb[i, N+1]), choosen_server) ) #(user, server) alloc pair
    print('RL Num of allocation: {}'.format(len(rl_allocation)))

    return rl_allocation

In [7]:
#Load model
def rl_algo_und():
    #for each server find the number of possible users using model
    server_capacity = np.zeros((N, S))
    for server_id in range(N):
        state = server_state[server_id]
        
        action = model_und.predict(np.array(state), deterministic=True)
        
        u1 = (action[0]//10)*2 + 1
        u2 = (action[0]%10)*2 + 1
        server_capacity[server_id][0] = u1 #model output
        server_capacity[server_id][1] = u2 #model output
    
    #sort with number of servers per user
    col1 = np.array([np.sum(ngb,axis=1)])
    col2 = np.array([np.arange(U)])
    sorted_ngb = np.concatenate((ngb, col1.T, col2.T), axis=1) #add rowsum and index column
    sorted_ngb = sorted_ngb[np.argsort(sorted_ngb[:, N])] #sort the rows based on rowsum column

    #allocation heuristic
    #run allocation algorithm
    rl_allocation = []

    for i in range(U):
        server_list = np.where(sorted_ngb[i, :N] == 1)[0] #get the list of server to which user is connected

        if len(server_list) == 0: #skip users with no server
            continue
        
        ser = int(service[i]) #which service user is requesting
        choosen_server = server_list[np.argmax(server_capacity[server_list, ser])] #find the id of choosen server
        
        
        if  server_capacity[choosen_server][ser] > 0: #assigning user to choosen_server
            server_capacity[choosen_server][ser] -= 1 #decrement the server capacity

            rl_allocation.append( (int(sorted_ngb[i, N+1]), choosen_server) ) #(user, server) alloc pair
    print('RL Num of allocation: {}'.format(len(rl_allocation)))

    return rl_allocation

In [8]:
#Load model
def rl_algo_act():
    #for each server find the number of possible users using model
    server_capacity = np.zeros((N, S))
    for server_id in range(N):
        state = server_state[server_id]
        
        action = model_act.predict(np.array(state), deterministic=True)
        
        u1 = (action[0]//5)*4 + 1 #25 action space
        u2 = (action[0]%5)*4 + 1
        server_capacity[server_id][0] = u1 #model output
        server_capacity[server_id][1] = u2 #model output
    
    #sort with number of servers per user
    col1 = np.array([np.sum(ngb,axis=1)])
    col2 = np.array([np.arange(U)])
    sorted_ngb = np.concatenate((ngb, col1.T, col2.T), axis=1) #add rowsum and index column
    sorted_ngb = sorted_ngb[np.argsort(sorted_ngb[:, N])] #sort the rows based on rowsum column

    #allocation heuristic
    #run allocation algorithm
    rl_allocation = []

    for i in range(U):
        server_list = np.where(sorted_ngb[i, :N] == 1)[0] #get the list of server to which user is connected

        if len(server_list) == 0: #skip users with no server
            continue
        
        ser = int(service[i]) #which service user is requesting
        choosen_server = server_list[np.argmax(server_capacity[server_list, ser])] #find the id of choosen server
        
        
        if  server_capacity[choosen_server][ser] > 0: #assigning user to choosen_server
            server_capacity[choosen_server][ser] -= 1 #decrement the server capacity

            rl_allocation.append( (int(sorted_ngb[i, N+1]), choosen_server) ) #(user, server) alloc pair
    print('RL Num of allocation: {}'.format(len(rl_allocation)))

    return rl_allocation

In [9]:
#Load model
def rl_algo_thres10():
    users_high = 500
    users_low = 100 #to calculate the users numbers from action
    users_res = 100
    #for each server find the number of possible users using model
    server_capacity = np.zeros((N, S))
    for server_id in range(N):
        state = server_state[server_id]
#         if model_type == 'lin':
        action = model_thres10.predict(np.array(state), deterministic=True)
#         if model_type == 'exp':
#             action = model_exp.predict(np.array(state), deterministic=True)
        u1 = action[0]//5 + 1
        u2 = (action[0]+1) - (u1-1)*5
        server_capacity[server_id][0] = u1*100 #model output
        server_capacity[server_id][1] = u2*100 #model output
    
    #sort with number of servers per user
    col1 = np.array([np.sum(ngb,axis=1)])
    col2 = np.array([np.arange(U)])
    sorted_ngb = np.concatenate((ngb, col1.T, col2.T), axis=1) #add rowsum and index column
    sorted_ngb = sorted_ngb[np.argsort(sorted_ngb[:, N])] #sort the rows based on rowsum column

    #allocation heuristic
    #run allocation algorithm
    rl_allocation = []

    for i in range(U):
        server_list = np.where(sorted_ngb[i, :N] == 1)[0] #get the list of server to which user is connected

        if len(server_list) == 0: #skip users with no server
            continue
        
        ser = int(service[i]) #which service user is requesting
        choosen_server = server_list[np.argmax(server_capacity[server_list, ser])] #find the id of choosen server
        
        
        if  server_capacity[choosen_server][ser] > 0: #assigning user to choosen_server
            server_capacity[choosen_server][ser] -= 1 #decrement the server capacity

            rl_allocation.append( (int(sorted_ngb[i, N+1]), choosen_server) ) #(user, server) alloc pair
    print('RL Num of allocation: {}'.format(len(rl_allocation)))

    return rl_allocation

### Nearest Ngb Greedy Algorithm

In [10]:
def greedy_algo():
    server_capacity = np.zeros(N)
    rl_allocation = []
    for user_id in range(U):
        server_ngb_list = np.where(ngb[user_id, :N] == 1)[0] #get the list of server to which user is connected
        if len(server_ngb_list) == 0: #ignore the users which are not under any servers
            continue 
    
        #find the distance to each users in the server_ngb_list
        dist_list = np.array([ server_ngb_list, [server.iloc[i]['geometry'].centroid.distance(user.iloc[user_id]['geometry']) for i in server_ngb_list] ])
        # sorted list of servers based on the distance from users
        sorted_distance_list = dist_list[ :, dist_list[1].argsort()]
        #get the list of servers arranged in least to max distance
        server_list = sorted_distance_list[0].astype(int)
        
        lat = 0
        for server_id in server_list:
            lat = gamma[server_id][int(service[user_id])]
            if server_capacity[server_id]+lat <= latency_threshold-network_latency[server_id]:
                server_capacity[server_id] += lat #increment the server_capacity of server
                rl_allocation.append( (user_id, server_id) ) #(user, server) alloc pair
                break
    
    print('Greedy-Ngb Num of allocation: {}'.format(len(rl_allocation)))
    return rl_allocation

## Test

In [11]:
# U = 500
# N = 50
# S = 2
# latency_threshold = 50
# ngb, user, server, service, server_service = ngb_matrix(U, N, S)
# server_state, gamma = generate_server_state(N)

## Result Dataset

In [12]:
#Result file details
# result_file = 'allocation_results/allocation_for_s50_thres30.csv'
result_file = 'allocation_results/allocation_for_threats_s30.csv'
column_names = ["user", "server", 
                "ilp_user", "ilp_time",
                "greedy_user", "greedy_time",
                "rl_act_user", "rl_act_time",
                "rl_und_user", "rl_und_time",
                "rl_prop_user", "rl_prop_time",
                "rl_thres10_user", "rl_thres10_time", 
               ]
result_user = pd.DataFrame(columns = column_names)
result_user.to_csv(result_file, index=False)
result_user = pd.read_csv(result_file)

# main()

In [13]:
latency_threshold = 10
N = 30
S = 2 #number of services
for U in range(100, 600, 100):
#     for N in range(20, 100, 20):
    for epoch in range(50):
        print("User:", U, 'Server:', N, 'Epoch:', epoch)
#             U = 50 #Number of users #Define U, N and ngb Matrix
#             N = 10 #Number of Servers
        ngb, user, server, service, server_service, network_latency = ngb_matrix(U, N, S) #generate server and user from EUA data #determine neighbourhood matrix
        server_state, gamma = generate_server_state(N) #assign state and gamma for each user

        #=======ILP starts
        start = 0
        stop = 0
        execution_time_ilp = 0
        start = timeit.default_timer()

        ilp_aloc = ilp_algo() #call ILP algorithm

        stop = timeit.default_timer()
        execution_time_ilp = stop - start
        #========ILP ends
        
        #=======Greedy starts
        start = 0
        stop = 0
        execution_time_greedy = 0
        start = timeit.default_timer()

        greedy_aloc = greedy_algo() #call ILP algorithm

        stop = timeit.default_timer()
        execution_time_greedy = stop - start
        #========Greedy ends

        #=======RL_action
        start = 0
        stop = 0
        execution_time_rl_act = 0
        start = timeit.default_timer()

        rl_act_aloc = rl_algo_act() #call ILP algorithm

        stop = timeit.default_timer()
        execution_time_rl_act = stop - start
        #========RL_linear ends
        
        #=======RL_under train
        start = 0
        stop = 0
        execution_time_rl_und = 0
        start = timeit.default_timer()

        rl_und_aloc = rl_algo_und() #call ILP algorithm

        stop = timeit.default_timer()
        execution_time_rl_und = stop - start
        #========RL_utrain ends
        
        #=======RL_propertrain
        start = 0
        stop = 0
        execution_time_rl_prop = 0
        start = timeit.default_timer()

        rl_prop_aloc = rl_algo_prop() #call ILP algorithm

        stop = timeit.default_timer()
        execution_time_rl_prop = stop - start
        #========RL_utrain ends
        
        
        #=======RL_thres10
        start = 0
        stop = 0
        execution_time_rl_thres10 = 0
        start = timeit.default_timer()

        rl_thres10_aloc = rl_algo_thres10() #call ILP algorithm

        stop = timeit.default_timer()
        execution_time_rl_thres10 = stop - start
        #========RL_ends

        #========Store results to file
        to_append = [U, N,
                     len(ilp_aloc), execution_time_ilp,
                     len(greedy_aloc), execution_time_greedy,
                     len(rl_act_aloc), execution_time_rl_act,
                     len(rl_und_aloc), execution_time_rl_und,
                     len(rl_prop_aloc), execution_time_rl_prop,
                     len(rl_thres10_aloc), execution_time_rl_thres10,
                    ] 
        dseries = pd.Series(to_append, index = result_user.columns)
        result_user = result_user.append(dseries, ignore_index=True)
        print("epoch:", epoch)
result_user.to_csv(result_file, index=False)

User: 100 Server: 30 Epoch: 0
ILP Allocated Num of Users: 45
Greedy-Ngb Num of allocation: 41
RL Num of allocation: 77
RL Num of allocation: 70
RL Num of allocation: 77
RL Num of allocation: 77
epoch: 0
User: 100 Server: 30 Epoch: 1
ILP Allocated Num of Users: 42
Greedy-Ngb Num of allocation: 41
RL Num of allocation: 50
RL Num of allocation: 49
RL Num of allocation: 50
RL Num of allocation: 50
epoch: 1
User: 100 Server: 30 Epoch: 2
ILP Allocated Num of Users: 40
Greedy-Ngb Num of allocation: 35
RL Num of allocation: 63
RL Num of allocation: 59
RL Num of allocation: 64
RL Num of allocation: 64
epoch: 2
User: 100 Server: 30 Epoch: 3
ILP Allocated Num of Users: 43
Greedy-Ngb Num of allocation: 42
RL Num of allocation: 70
RL Num of allocation: 67
RL Num of allocation: 70
RL Num of allocation: 70
epoch: 3
User: 100 Server: 30 Epoch: 4
ILP Allocated Num of Users: 51
Greedy-Ngb Num of allocation: 43
RL Num of allocation: 75
RL Num of allocation: 72
RL Num of allocation: 75
RL Num of allocatio

ILP Allocated Num of Users: 44
Greedy-Ngb Num of allocation: 41
RL Num of allocation: 67
RL Num of allocation: 66
RL Num of allocation: 67
RL Num of allocation: 67
epoch: 40
User: 100 Server: 30 Epoch: 41
ILP Allocated Num of Users: 43
Greedy-Ngb Num of allocation: 38
RL Num of allocation: 67
RL Num of allocation: 63
RL Num of allocation: 67
RL Num of allocation: 67
epoch: 41
User: 100 Server: 30 Epoch: 42
ILP Allocated Num of Users: 39
Greedy-Ngb Num of allocation: 35
RL Num of allocation: 49
RL Num of allocation: 49
RL Num of allocation: 49
RL Num of allocation: 49
epoch: 42
User: 100 Server: 30 Epoch: 43
ILP Allocated Num of Users: 40
Greedy-Ngb Num of allocation: 37
RL Num of allocation: 63
RL Num of allocation: 57
RL Num of allocation: 64
RL Num of allocation: 64
epoch: 43
User: 100 Server: 30 Epoch: 44
ILP Allocated Num of Users: 50
Greedy-Ngb Num of allocation: 46
RL Num of allocation: 72
RL Num of allocation: 69
RL Num of allocation: 72
RL Num of allocation: 72
epoch: 44
User: 

ILP Allocated Num of Users: 52
Greedy-Ngb Num of allocation: 48
RL Num of allocation: 122
RL Num of allocation: 114
RL Num of allocation: 122
RL Num of allocation: 122
epoch: 30
User: 200 Server: 30 Epoch: 31
ILP Allocated Num of Users: 61
Greedy-Ngb Num of allocation: 53
RL Num of allocation: 128
RL Num of allocation: 113
RL Num of allocation: 128
RL Num of allocation: 128
epoch: 31
User: 200 Server: 30 Epoch: 32
ILP Allocated Num of Users: 63
Greedy-Ngb Num of allocation: 51
RL Num of allocation: 130
RL Num of allocation: 113
RL Num of allocation: 132
RL Num of allocation: 132
epoch: 32
User: 200 Server: 30 Epoch: 33
ILP Allocated Num of Users: 63
Greedy-Ngb Num of allocation: 54
RL Num of allocation: 143
RL Num of allocation: 130
RL Num of allocation: 143
RL Num of allocation: 143
epoch: 33
User: 200 Server: 30 Epoch: 34
ILP Allocated Num of Users: 56
Greedy-Ngb Num of allocation: 48
RL Num of allocation: 105
RL Num of allocation: 98
RL Num of allocation: 105
RL Num of allocation: 1

ILP Allocated Num of Users: 62
Greedy-Ngb Num of allocation: 57
RL Num of allocation: 176
RL Num of allocation: 151
RL Num of allocation: 183
RL Num of allocation: 183
epoch: 20
User: 300 Server: 30 Epoch: 21
ILP Allocated Num of Users: 69
Greedy-Ngb Num of allocation: 56
RL Num of allocation: 198
RL Num of allocation: 184
RL Num of allocation: 201
RL Num of allocation: 201
epoch: 21
User: 300 Server: 30 Epoch: 22
ILP Allocated Num of Users: 65
Greedy-Ngb Num of allocation: 55
RL Num of allocation: 200
RL Num of allocation: 176
RL Num of allocation: 203
RL Num of allocation: 203
epoch: 22
User: 300 Server: 30 Epoch: 23
ILP Allocated Num of Users: 70
Greedy-Ngb Num of allocation: 52
RL Num of allocation: 205
RL Num of allocation: 158
RL Num of allocation: 209
RL Num of allocation: 209
epoch: 23
User: 300 Server: 30 Epoch: 24
ILP Allocated Num of Users: 62
Greedy-Ngb Num of allocation: 53
RL Num of allocation: 205
RL Num of allocation: 171
RL Num of allocation: 216
RL Num of allocation: 

ILP Allocated Num of Users: 70
Greedy-Ngb Num of allocation: 58
RL Num of allocation: 251
RL Num of allocation: 213
RL Num of allocation: 265
RL Num of allocation: 265
epoch: 10
User: 400 Server: 30 Epoch: 11
ILP Allocated Num of Users: 68
Greedy-Ngb Num of allocation: 51
RL Num of allocation: 238
RL Num of allocation: 217
RL Num of allocation: 257
RL Num of allocation: 257
epoch: 11
User: 400 Server: 30 Epoch: 12
ILP Allocated Num of Users: 69
Greedy-Ngb Num of allocation: 53
RL Num of allocation: 271
RL Num of allocation: 236
RL Num of allocation: 296
RL Num of allocation: 296
epoch: 12
User: 400 Server: 30 Epoch: 13
ILP Allocated Num of Users: 71
Greedy-Ngb Num of allocation: 56
RL Num of allocation: 225
RL Num of allocation: 198
RL Num of allocation: 225
RL Num of allocation: 225
epoch: 13
User: 400 Server: 30 Epoch: 14
ILP Allocated Num of Users: 72
Greedy-Ngb Num of allocation: 59
RL Num of allocation: 270
RL Num of allocation: 221
RL Num of allocation: 286
RL Num of allocation: 

ILP Allocated Num of Users: 67
Greedy-Ngb Num of allocation: 52
RL Num of allocation: 279
RL Num of allocation: 205
RL Num of allocation: 298
RL Num of allocation: 298
epoch: 0
User: 500 Server: 30 Epoch: 1
ILP Allocated Num of Users: 66
Greedy-Ngb Num of allocation: 53
RL Num of allocation: 285
RL Num of allocation: 262
RL Num of allocation: 304
RL Num of allocation: 304
epoch: 1
User: 500 Server: 30 Epoch: 2
ILP Allocated Num of Users: 67
Greedy-Ngb Num of allocation: 53
RL Num of allocation: 295
RL Num of allocation: 252
RL Num of allocation: 341
RL Num of allocation: 341
epoch: 2
User: 500 Server: 30 Epoch: 3
ILP Allocated Num of Users: 65
Greedy-Ngb Num of allocation: 51
RL Num of allocation: 299
RL Num of allocation: 261
RL Num of allocation: 347
RL Num of allocation: 347
epoch: 3
User: 500 Server: 30 Epoch: 4
ILP Allocated Num of Users: 70
Greedy-Ngb Num of allocation: 57
RL Num of allocation: 295
RL Num of allocation: 254
RL Num of allocation: 328
RL Num of allocation: 328
epoc

ILP Allocated Num of Users: 67
Greedy-Ngb Num of allocation: 52
RL Num of allocation: 251
RL Num of allocation: 216
RL Num of allocation: 266
RL Num of allocation: 266
epoch: 40
User: 500 Server: 30 Epoch: 41
ILP Allocated Num of Users: 68
Greedy-Ngb Num of allocation: 57
RL Num of allocation: 275
RL Num of allocation: 221
RL Num of allocation: 301
RL Num of allocation: 301
epoch: 41
User: 500 Server: 30 Epoch: 42
ILP Allocated Num of Users: 74
Greedy-Ngb Num of allocation: 64
RL Num of allocation: 289
RL Num of allocation: 227
RL Num of allocation: 321
RL Num of allocation: 321
epoch: 42
User: 500 Server: 30 Epoch: 43
ILP Allocated Num of Users: 70
Greedy-Ngb Num of allocation: 55
RL Num of allocation: 296
RL Num of allocation: 215
RL Num of allocation: 327
RL Num of allocation: 327
epoch: 43
User: 500 Server: 30 Epoch: 44
ILP Allocated Num of Users: 74
Greedy-Ngb Num of allocation: 56
RL Num of allocation: 254
RL Num of allocation: 235
RL Num of allocation: 293
RL Num of allocation: 

In [14]:
result_user

,user,server,ilp_user,ilp_time,greedy_user,greedy_time,rl_act_user,rl_act_time,rl_und_user,rl_und_time,rl_prop_user,rl_prop_time,rl_thres10_user,rl_thres10_time
0,100.0,30.0,45.0,6.897818,41.0,0.026541,77.0,0.167978,70.0,0.012435,77.0,0.019812,77.0,0.012341
1,100.0,30.0,42.0,0.140814,41.0,0.020789,50.0,0.011666,49.0,0.011208,50.0,0.011194,50.0,0.011300
2,100.0,30.0,40.0,0.167810,35.0,0.025199,63.0,0.011811,59.0,0.011184,64.0,0.011244,64.0,0.011242
3,100.0,30.0,43.0,0.164766,42.0,0.026608,70.0,0.011779,67.0,0.011254,70.0,0.011260,70.0,0.011286
4,100.0,30.0,51.0,0.149066,43.0,0.025358,75.0,0.011712,72.0,0.011280,75.0,0.011247,75.0,0.011284
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,500.0,30.0,72.0,0.747710,59.0,0.127495,311.0,0.014970,265.0,0.014544,355.0,0.014694,355.0,0.014674
246,500.0,30.0,72.0,0.840029,60.0,0.127422,299.0,0.014259,246.0,0.013873,318.0,0.014070,318.0,0.014003
247,500.0,30.0,69.0,0.784908,51.0,0.125308,278.0,0.014771,246.0,0.014326,318.0,0.014450,318.0,0.014440
248,500.0,30.0,71.0,0.763599,57.0,0.117507,282.0,0.014226,212.0,0.014077,300.0,0.014254,300.0,0.014328
